In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import community
import sys
import math


Bad key "text.kerning_factor" on line 4 in
C:\Users\Yueshuwei Wu\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
# Read the original edge list 
G = nx.read_edgelist('users edge list.txt',comments='#', delimiter=None, create_using=nx.DiGraph, nodetype=None, encoding='utf-8')
print (nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 100386
Number of edges: 2286592
Average in degree:  22.7780
Average out degree:  22.7780


In [4]:
# Read information for nodes
data_s=[]
with open("ver. 4 users_neighborhood_anon.csv","r") as file:
    for line in file:
        data = line.strip().split(",")
        data_s.append(data)
tag =data_s[0]

users=[]
for data in data_s[1:]:
    user=dict(zip(tag,data))
    users.append(user)

In [44]:
print(data_s[0])
print(data_s[2])
print(data_s[5])

['user_id', 'hate', 'statuses_count', 'followers_count', 'followees_count', 'favorites_count', 'listed_count', 'sentiment', 'subjectivity', 'number hashtags', 'tweet number', 'retweet number', 'quote number', 'status length', 'number urls', 'baddies', 'mentions']
['1', 'other', '2352', '19609', '309', '61', '197', '0.088142078', '0.418649474', '40', '199', '0', '0', '101.7135678', '20', '10', '6']
['4', 'other', '1998', '17643', '19355', '485', '239', '0.121533267', '0.435334321', '1710', '101', '99', '0', '152.175', '198', '35', '7']


In [21]:
df = pd.read_excel('Community Id List.XLSX') # can also index sheet by name or fetch all sheets
conservatives = df['Conservative'].tolist()
c_users=[]
for user in users:
    if float(user['user_id']) in conservatives:
        c_users.append(user)



In [22]:
c_nodes=[]
c_edges=[]

for user in c_users:
    c_nodes.append(user['user_id'])


edges=[]
with open("users edge list.txt","r") as file:
    for line in file:
        edge = line.strip().split(" ")
        edges.append(edge)

# we only need the retweet network of our target users,which is the edge coming in
# also get filter out the self-loops
for edge in edges:
    if ((edge[1] in c_nodes))and (edge[0]!=edge[1]):
        c_edges.append(edge)

#Save the edge list for future use
with open("C Edges.csv","w") as file:
    for e in c_edges:
        file.write(e[0]+','+e[1]+'\n')

In [25]:
#Generate the node list for C network
with open("C Nodes.csv","w") as file:
    file.write('user_id,hate,statuses_count,followers_count,followees_count,favorites_count,listed_count,sentiment,subjectivity,number hashtags,tweet number,retweet number,quote number,status length,number urls,baddies,mentions\n')
    for user in users:
        for e in c_edges:
            if (user['user_id']in e):
                file.write(user['user_id']+','+user['hate']+','+user['statuses_count']+','+user['followers_count']+','+user['followees_count']+','+user['favorites_count']+','+user['listed_count']+','+user['sentiment']+','+user['subjectivity']+','+user['number hashtags']+','+user['tweet number']+','+user['retweet number']+','+user['quote number']+','+user['status length']+','+user['number urls']+','+user['baddies']+','+user['mentions']+'\n')
                break
    

In [45]:
#Divide the users into differnt groups
h_users=[]
n_users=[]
o_users=[]

for user in users:
    if user['hate']=='hateful':
            h_users.append(user)
    if user['hate']=='normal':
            n_users.append(user)
    if user['hate']=='other':
            o_users.append(user)

print(len(h_users))
print(len(n_users))
print(len(o_users))

544
4427
95413


In [47]:
#Generate the list and edge file for our target users
hn_nodes=[]
hn_edges=[]

for user in h_users:
    hn_nodes.append(user['user_id'])
for user in n_users:
    hn_nodes.append(user['user_id'])
    
    
edges=[]
with open("users edge list.txt","r") as file:
    for line in file:
        edge = line.strip().split(" ")
        edges.append(edge)

# we only need the retweet network of our target users,which is the edge coming in
# also get filter out the self-loops
for edge in edges:
    if ((edge[1] in hn_nodes))and (edge[0]!=edge[1]):
        hn_edges.append(edge)

#Save the edge list for future use
with open("HN Edges.csv","w") as file:
    for e in hn_edges:
        file.write(e[0]+','+e[1]+'\n')

In [64]:
#Generate the node list for HN network
with open("HN Nodes.csv","w") as file:
    file.write('user_id,hate,statuses_count,followers_count,followees_count,favorites_count,listed_count,sentiment,subjectivity,number hashtags,tweet number,retweet number,quote number,status length,number urls,baddies,mentions\n')
    for user in users:
        for e in hn_edges:
            if (user['user_id']in e):
                file.write(user['user_id']+','+user['hate']+','+user['statuses_count']+','+user['followers_count']+','+user['followees_count']+','+user['favorites_count']+','+user['listed_count']+','+user['sentiment']+','+user['subjectivity']+','+user['number hashtags']+','+user['tweet number']+','+user['retweet number']+','+user['quote number']+','+user['status length']+','+user['number urls']+','+user['baddies']+','+user['mentions']+'\n')
                break
    

In [49]:
#Generate the list and edge file for hateful users
h_nodes=[]
h_edges=[]

for user in h_users:
    h_nodes.append(user['user_id'])

# we only need the retweet network of our target users,which is the edge coming in
# also get filter out the self-loops
for edge in edges:
    if ((edge[1] in h_nodes))and (edge[0]!=edge[1]):
        h_edges.append(edge)

#Save the edge list for future use
with open("Hateful Edges.csv","w") as file:
    for e in h_edges:
        file.write(e[0]+','+e[1]+'\n')

In [65]:
#Generate the node list for H network
with open("H Nodes.csv","w") as file:
    file.write('user_id,hate,statuses_count,followers_count,followees_count,favorites_count,listed_count,sentiment,subjectivity,number hashtags,tweet number,retweet number,quote number,status length,number urls,baddies,mentions\n')
    for user in users:
        for e in h_edges:
            if (user['user_id']in e):
                file.write(user['user_id']+','+user['hate']+','+user['statuses_count']+','+user['followers_count']+','+user['followees_count']+','+user['favorites_count']+','+user['listed_count']+','+user['sentiment']+','+user['subjectivity']+','+user['number hashtags']+','+user['tweet number']+','+user['retweet number']+','+user['quote number']+','+user['status length']+','+user['number urls']+','+user['baddies']+','+user['mentions']+'\n')
                break
       

In [51]:
#Generate the list and edge file for normal users
n_nodes=[]
n_edges=[]

for user in n_users:
    n_nodes.append(user['user_id'])

# we only need the retweet network of our target users,which is the edge coming in
# also get filter out the self-loops
for edge in edges:
    if ((edge[1] in n_nodes))and (edge[0]!=edge[1]):
        n_edges.append(edge)

#Save the edge list for future use
with open("Normal Edges.csv","w") as file:
    for e in n_edges:
        file.write(e[0]+','+e[1]+'\n')

In [66]:
#Generate the node list for N network
with open("N Nodes.csv","w") as file:
    file.write('user_id,hate,statuses_count,followers_count,followees_count,favorites_count,listed_count,sentiment,subjectivity,number hashtags,tweet number,retweet number,quote number,status length,number urls,baddies,mentions\n')
    for user in users:
        for e in n_edges:
            if (user['user_id']in e):
                file.write(user['user_id']+','+user['hate']+','+user['statuses_count']+','+user['followers_count']+','+user['followees_count']+','+user['favorites_count']+','+user['listed_count']+','+user['sentiment']+','+user['subjectivity']+','+user['number hashtags']+','+user['tweet number']+','+user['retweet number']+','+user['quote number']+','+user['status length']+','+user['number urls']+','+user['baddies']+','+user['mentions']+'\n')
                break
    
    

In [27]:
#Check the number of edges we got
print(len(hn_edges))
print(len(h_edges))
print(len(n_edges))
print(len(h_edges)+len(n_edges)-len(hn_edges))

123824
16024
107800
0


In [ ]:
# Generate our HN network,
Ghn = nx.read_edgelist('HN Edges.txt', comments='#', delimiter=None, create_using=nx.DiGraph, nodetype=None, encoding='utf-8')
print (nx.info(Ghn))

In [ ]:
#Give the nodes differnt color
color_map = []
for node in Ghn:
    if node in h_nodes:
        color_map.append('gold')
    elif node in n_nodes:
        color_map.append('cornflowerblue')
    else:
        color_map.append('limegreen')